# Configuration 

In [1]:
import easydict
from downstream_modules.data_utils import name_to_class

args = easydict.EasyDict(
    {
        # 'model_path': './artifacts/pmg_uni/epochs=20',
        'model_path': './artifacts/pmg_multi/epochs=20',
        # 'model_path': './artifacts/pmg_uni/baseline1_dirt_epochs=20',
        'arch': 'pmg', # 'pmg', 'resnet'
        'method': 'multi', # 'uni', 'multi', 'baseline'
        'dataset': 'sofar_v3',
        'data_root_path': '../../../dataset/',
        'train_class': 'outer_normal,outer_damage,outer_dirt,outer_wash,inner_wash,inner_dashboard,inner_cupholder,inner_glovebox,inner_washer_fluid,inner_rear_seat,inner_sheet_dirt', 
        'test_class': 'outer_normal,outer_damage,outer_dirt,outer_wash,inner_wash,inner_dashboard,inner_cupholder,inner_glovebox,inner_washer_fluid,inner_rear_seat,inner_sheet_dirt', 
        # 'train_class': 'outer_normal,outer_dirt',
        # 'test_class': 'outer_normal,outer_dirt',

        'num_workers': 4, 
        'batch_size': 128 ,
        'ce_label': False, 
        'show_img': False, # to show result imgs 
    }
)
args.train_class_name = [item for item in args.train_class.split(',')]
args.test_class_name = [item for item in args.test_class.split(',')]

args.train_class = [name_to_class[item] for item in args.train_class.split(',')]
args.test_class = [name_to_class[item] for item in args.test_class.split(',')]

# Dataloader 

In [2]:
from downstream_modules.data_utils import create_dataloader

_, test_loader = create_dataloader(args)

# Model

In [3]:
import os 
import torch 
import torch.nn as nn 

if args.arch == 'resnet':
    pass 
# TODO: 
#     import torchvision
    
#     model = torchvision.models.resnet50(pretrained=False)
#     in_feats = model.fc.in_features
#     model.fc = nn.Identity()

elif args.arch == 'pmg':
    from models.resnet import resnet50
    from models.pmg import PMG, PMG_Multi
    
    model = resnet50(pretrained=False)
    ## TODO: uni, multi case 
    if args.method == 'uni' or args.method == 'baseline':
        model = PMG(model, feature_size = 512, num_classes = len(args.train_class))
    elif args.method == 'multi':
        model = PMG(model, feature_size = 512, num_classes = len(args.train_class)-1) 
        # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        # model = PMG_Multi(model, 512, device)


state_dict = torch.load(os.path.join(args.model_path, 'last.pth'), map_location='cpu')
model.load_state_dict(state_dict)
print('pre-trained v2 model is loaded')

pre-trained v2 model is loaded


# Test

In [10]:
from tqdm import tqdm 
import numpy as np 

import torch.nn.functional as F 

@torch.no_grad() 
def ood_test(dataloader, model):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    model.to(device)
    model.eval()

    test_preds = []
    test_labels =[] 

    for (img, label) in tqdm(dataloader):
        img = img.to(device)

        # forward
        if args.arch == 'resnet':
            out = model(img)
            
        elif args.arch == 'pmg':
            out = model._forward(img)
            out = out[-1]
        
        # pred 
        if args.method == 'uni':
            out = F.softmax(out,dim=1)
            if args.task == 'defect':
                pred = out[:,1]
            elif args.task == 'dirt':
                pred = out[:,2]
            test_preds.extend(pred.view(-1).cpu().detach().numpy().tolist())
        elif args.method == 'baseline':
            out = F.softmax(out, dim=1)
            pred = out[:,1]
            test_preds.extend(pred.view(-1).cpu().detach().numpy().tolist())
        elif args.method == 'multi':
            if args.task == 'defect':
                pred = F.sigmoid(out[:,0])
                # pred = F.kl_div(pred, torch.ones_like(pred) / 2., reduction='none')
                # pred = torch.abs(pred - torch.ones_like(pred) / 2.)
                # pred = torch.max(out[0],dim=1)[0]
            elif args.task == 'dirt':
                pred = F.sigmoid(out[:,1])
                # pred = F.kl_div(pred, torch.ones_like(pred) / 2., reduction='none')
                # pred = torch.abs(pred - torch.ones_like(pred) / 2.)
                # pred = torch.max(out[1],dim=1)[0]
            
            test_preds.extend(pred.view(-1).cpu().detach().numpy().tolist())

        test_labels.extend(label.view(-1).cpu().numpy().tolist())
    
    if args.task == 'dirt':
        target = [2]
    elif args.task == 'defect':
        target = [1]
    task_idx = np.isin(test_labels, target)

    test_labels = np.concatenate(([1 for _ in range(len(np.array(test_preds)[task_idx]))],[0 for _ in range(len(np.array(test_preds)[~task_idx]))]))    
    test_preds = np.concatenate((np.array(test_preds)[task_idx], np.array(test_preds)[~task_idx]))
    
    return test_labels, test_preds

In [11]:
from sklearn.metrics import * 
tasks = ['dirt', 'defect']
for task in tasks:
    args.task = task
    labels, preds = ood_test(test_loader, model)
    print(task, roc_auc_score(labels, preds))

  0%|          | 0/8 [00:00<?, ?it/s]

torch.Size([10])


/home/jovyan/.local/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
 12%|█▎        | 1/8 [00:12<01:30, 12.97s/it]

torch.Size([10])


 25%|██▌       | 2/8 [00:14<00:36,  6.07s/it]

...

torch.Size([10])


100%|██████████| 8/8 [00:31<00:00,  3.90s/it]


dirt 0.9381674346916629


  0%|          | 0/8 [00:00<?, ?it/s]/home/jovyan/.local/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 8/8 [00:31<00:00,  3.91s/it]

defect 0.8734478672985783
